## docker

```sh
docker search <name>  # Поиск образа на docker hub
docker pull <name>    # Скачать образ по имени
```

```sh
docker images       # Список образов
docker images name  # Список образов по имени
```

```sh
docker run --name MyCont            # Установкить Имя контейнера
           -p 9000:8080             # Внешний:Внутренний Порты
           -v /path/to/folder:/www  # Пробросить директорию в контейнер
           -d                       # Detached
           -it bash                 # Подключиться к запущенному контейнеру
           --rm                     # Удалить после завершения
           name                     # image name
```

```sh
docker exec -it MyCont bash         # Подключиттся и запустить bash
```

```sh
docker attach name  # Подключиться к запущенному контейнеру
docker npm run      # Выполнить команду
```

```sh
docker cp file.txt cont:/path/to/dest      # Копирование в контейнер
docker cp cont:/path/to/file ./local_file  # Копирование из контейнера
```

```sh
docker stop name     # Остановить контейнер
docker start name    # Запустить остановленный контейнер
docker restart name  # Перезапуск контейнера
```

```sh
docker ps       # Список запущенных контейнеров
          -a    # Список недавно запущенных контейнеров
          -l    # Последний запущенный контейнер
          -n 1  # Показать последний 1 контейнер
```

```sh
docker rm MyCont                   # Удалить Контейнер
docker rm -f $(docker ps -aq)      # Удалить все контейнеры
docker rmi MyCont                  # Удалить Образ
docker rmi -f $(docker images -q)  # Удалить все Образы
docker system prune -a             # Удалить все
```

```sh
docker logs MyCont            # Посмотреть stdout контейнера
docker logs -f MyCont         # Подключиться к stdout контейнера
docker logs --tail 10 MyCont  # Вывести 10 последних записей stdout
docker logs -t MyCont         # Вывести каждую запись со временем
docker top MyCont             # Посмотреть процессы контейнера
docker inspect MyCont         # Информация о контейнере
docker inspect --format='{{ .State.Running }}' MyCont
```

```sh
docker commit MyCont newMyCont              # Коммит контейнера
docker commit -m="message" --author="Name"  # Доп. информация
```

## Dockerfile
### Запуск
```bash
docker build -t="name:tag" -f ./path/to/dockerfile
```

### Слои
```bash
# Базовый образ
FROM python:3.11-slim             # Указание родительского образа
FROM alpine:latest AS stage       # Многостадийная сборка

# Автор образа
LABEL maintainer="email"          # Современный способ

# Переменные сборки
ARG VERSION=1.0                   # Аргумент, доступен только во время сборки
ENV APP_ENV=prod                  # Переменная окружения (доступна всегда)
ENV PYTHONUNBUFFERED=1            # Отключить буферизацию вывода
ENV PATH="/app/.venv/bin:$PATH"   # Добавить путь в окружение

# Обновление и установка
RUN apt update && apt upgrade -y  # Обновление пакетов
RUN apt install -y curl git vim   # Установка ПО

# Очистка
RUN rm -rf /var/lib/apt/lists/*   # Если надо почистить контейнер

# Рабочая директория
WORKDIR /app                      # Установить рабочую директорию
VOLUME /data                      # Объявление точки монтирования

# Установка Python-зависимостей
COPY requirements.txt .           # Копируем файл зависимостей
RUN pip install --no-cache-dir -r requirements.txt  # Установка зависимостей

# Копирование файлов
COPY . /app/                      # Копирование всего проекта
ADD file.tar.gz /data/            # Копирование с автораспаковкой архива
ADD https://example.com/file.txt /tmp/file.txt  # Загрузка с URL

# Установка прав
RUN chmod +x entrypoint.sh        # Сделать файл исполняемым
RUN chown -R user:user /app       # Изменить владельца

# Пользователь
USER appuser                      # Запуск под другим пользователем

# Порты
EXPOSE 80                         # Документально открыть порт
EXPOSE 5000/tcp                   # Явно указать TCP-порт

# Команды запуска
CMD ["python", "app.py"]          # Команда по умолчанию
ENTRYPOINT ["./start.sh"]         # Основная команда, CMD — аргументы к ней
CMD ["--host=0.0.0.0"]            # Аргументы к entrypoint
SHELL ["/bin/bash", "-c"]         # Установить shell по умолчанию

# События сборки
ONBUILD COPY . /src               # Выполняется в дочернем образе
ONBUILD RUN npm install           # Подготовка для наследования

# Проверка состояния контейнера
HEALTHCHECK --interval=30s CMD curl -f http://localhost/ || exit 1

# Сигнал завершения
STOPSIGNAL SIGTERM                # Задать сигнал остановки
```

### Многостадийная сборка (build → release)
```sh
FROM node:18 AS builder
WORKDIR /app
COPY . .
RUN npm run build

FROM nginx:alpine
COPY --from=builder /app/dist /usr/share/nginx/html
```

## Пуш контейнеров
```bash
docker login   # Логин
docker logout  # Выход
docker tag cont:v2 kraspy/cont:v2  # Переименовать контейнер
docker push cont:v2 kraspy/cont:v2 # Пуш контейнера на docker hub
```

## Examples
```bash
# Tags
docker build -t myapp:1.0 .
docker push myapp:1.0

# Volumes
docker run -d -v my_volume:/data --name name my_image

# Network
docker network create my_network
docker run -d --network=my_network --name name my_image

# Resources
docker run -d --name name --memory=500m --cpus=1.0 my_image
```